In [5]:
import getGmail_Class
import os, sys, logging
from tqdm import trange
from datetime import datetime
import pandas as pd
import re

In [6]:
gGC = getGmail_Class.gmailService()

In [7]:
Num, Name, investment_company, Date, Path, ID = [[] for i in range(6)]

# Create log file
if not os.path.isdir("log"):
    os.mkdir("log")

FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level = logging.INFO, filename = "./log/" + datetime.now().strftime("%Y_%m_%d") + '.log', filemode = 'w', format = FORMAT)
logging.info('Updating email start')

# request a list of all the messages
result = gGC.service.users().messages().list(userId = 'me', maxResults = 500, labelIds = ["INBOX"]).execute()
messages = result.get('messages')

# get mail ID from messages
for i in range(len(messages)):
    ID.append(messages[i]['id'])

if len(ID) == 0:
    logging.info('Inbox quantity is 0')
    logging.info('Updating email end')
    sys.exit(0)

# iterate through all the messages
for i in trange(len(ID)):
    # Get the message from its id
    txt = gGC.service.users().messages().get(userId = 'me', id = ID[i]).execute()
    payload = txt['payload']
    headers = payload['headers']
    date = gGC.getDate(headers)
    subject = gGC.verifySubject(headers)

    if len(subject) != 0:
        tempNum, tempName, tempInvestment_company, tempPath = gGC.getResearch_report(headers, subject, payload, ID[i])

        if len(tempNum) != 0:
            for j in range(len(tempNum)):
                Num.append(tempNum[j])
                Name.append(tempName[j])
                Date.append(date)
                investment_company.append(tempInvestment_company)
                Path.append(tempPath[j])
#                     # Modify labels
#                     gGC.modifyLabels(ID[i], "Label2")
    else:
        # Modify labels
        gGC.modifyLabels(ID[i], "Label3")

df = pd.DataFrame({ "Number" : Num, "Name" : Name, "Investment company" : investment_company, "Date" : Date, "File path" : Path })

logging.info('Updating email end')

100%|█████████████████████████████████████████| 500/500 [06:38<00:00,  1.26it/s]


In [8]:
df

,Number,Name,Investment company,Date,File path
0,2892,第一金,統一投顧,2022_3_03,./file/2892/統一投顧_第一金_統一投顧-陳晏平-【投資速報】第一金 (2892....
1,2892,第一金,統一投顧,2021_12_01,./file/2892/統一投顧_第一金_統一投顧-陳晏平-【投資速報】第一金 (2892....
2,2892,第一金,統一投顧,2021_6_01,./file/2892/統一投顧_第一金_統一投顧-陳晏平-【投資速報】第一金 (2892....
3,2887,台新金,統一投顧,2021_5_20,./file/2887/統一投顧_台新金_統一投顧-陳晏平-【投資速報】台新金 (2887....
4,2892,第一金,統一投顧,2020_12_01,./file/2892/統一投顧_第一金_統一投顧-陳晏平-【投資速報】第一金 (2892....
...,...,...,...,...,...
249,9939,宏全,,2020_7_27,./file/9939/宏全_20200727-個股報告-宏全(9939).pdf
250,3035,智原,,2020_7_27,./file/3035/智原_20200727-個股報告-智原(3035).pdf
251,2520,冠德,,2020_7_27,./file/2520/冠德_20200727-個股報告-冠德(2520).pdf
252,3035,智原,元富,2020_7_27,./file/3035/元富_智原_智原(3035)-20200727-速報.pdf
